In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.003
N = 61
times = range(0, dt * (N - 1), length=N)
k_trans = 21
n = 15
m = 5

# Initial condition. Currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1] = -lb / 2.5                # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
xinit[3] = -20 * pi / 180           # theta
xinit[6] = -lb                      # x2
xinit[7] = 0.2                      # y2
xinit[9] = -3.0                     # yb_dot
xinit[14] = -3.0                    # y2_dot
init_mode = 1

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

# # Initial condition. Currently, we assume the initial mode ID is 2
# xinit = zeros(n)
# xinit[1] = lb / 2.5                 # xb
# xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
# xinit[3] = 20 * pi / 180            # theta
# xinit[4] = lb                       # x1
# xinit[5] = 0.2                      # y1
# xinit[9] = -3.0                     # yb_dot
# xinit[12] = -3.0                    # y1_dot
# init_mode = 2

# # Desired final state
# xterm = zeros(n)
# xterm[1] = lb / 2            # xb
# xterm[2] = sqrt(l1^2 + l2^2) # yb
# xterm[4] = lb;               # x1

In [4]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, init_mode, dt);

In [6]:
# Objective
Q = Diagonal([10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal(fill(1e-3, 5))
R[end, end] = 0.0
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [8]:
# Initial guess
Random.seed!(1)

# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

# initialize Xguess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:14] = xterm[1:14]
    end
    
    Xguess[k][end] = dt * (k - 1)
end

# initialize Uguess
Uguess = [zeros(m) + rand(m) for u in Uref]
for k = 1: N-1
    Uguess[k][end] = dt
end

In [9]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1252665
Number of nonzeros in inequality constraint Jacobian.:    74115
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1215
                     variables with only lower bounds:       80
                variables with lower and upper bounds:      121
                     variables with only upper bounds:        0
Total number of equal

([-0.19999999999982299, 0.45355339059324984, -0.34906585039887045, 1.8706891456668184e-15, -2.8698911078301724e-16, -0.50000000000002, 0.19999999999999998, 6.206102426150905e-15, -3.0, -1.5179324866670983e-16  …  -0.49999999999995803, 1.0228356689120755e-16, 4.976380481789923e-14, -8.175065651269594e-15, -1.3052056714401468e-15, -5.357867333657261e-16, -5.3255544840558395e-20, -4.253734407611642e-15, -2.281701181094065e-19, 0.294035717515678], Ipopt.Optimizer)

In [10]:
Z_sol[1:15] - xinit

15-element Vector{Float64}:
  1.770250612764812e-13
 -2.398081733190338e-14
 -4.551914400963142e-15
  1.8706891456668184e-15
 -2.8698911078301724e-16
 -1.9984014443252818e-14
 -2.7755575615628914e-17
  6.206102426150905e-15
  0.0
 -1.5179324866670983e-16
  2.927394742331067e-16
 -2.447119847394965e-18
 -1.2687496169002677e-15
  0.0
 -6.048270491533398e-18

In [11]:
Z_sol[end-14:end] - xterm

15-element Vector{Float64}:
 -2.0727863869751673e-13
  2.992051051364797e-14
  4.628051908835679e-15
  6.384408340457702e-15
  1.0242254170264418e-16
  4.196643033083092e-14
  1.0228356689120755e-16
  4.976380481789923e-14
 -8.175065651269594e-15
 -1.3052056714401468e-15
 -5.357867333657261e-16
 -5.3255544840558395e-20
 -4.253734407611642e-15
 -2.281701181094065e-19
  0.294035717515678

In [26]:
a = zeros(N-1)
for k = 1:N-1
    vb_x_k = Z_sol[8+20*(k-1)]
    vb_x_next = Z_sol[8+20*(k-1+1)]
    dt = Z_sol[20+20*(k-1)]
    a[k] = (vb_x_next - vb_x_k) / dt
    ay_dyn = contact1_dynamics(model, Z_sol[1+20*(k-1):14+20*(k-1)], Z_sol[16+20*(k-1):19+20*(k-1)])
    @show ay_dyn[9]
end

ay_dyn[9] = 21.38022099172219
ay_dyn[9] = 23.016862762981056
ay_dyn[9] = 20.405150710981275
ay_dyn[9] = 17.914325798146244
ay_dyn[9] = 15.778165390059376
ay_dyn[9] = 14.177164957496862
ay_dyn[9] = -16.152413540378515
ay_dyn[9] = 12.097765565541247
ay_dyn[9] = 14.286639822539628
ay_dyn[9] = 12.497462012074903
ay_dyn[9] = 7.949102915863618
ay_dyn[9] = 13.38911166380892
ay_dyn[9] = 10.637991666501728
ay_dyn[9] = 11.455520002647587
ay_dyn[9] = 11.578223256625089
ay_dyn[9] = 10.744649475991563
ay_dyn[9] = 11.284703890974681
ay_dyn[9] = 11.506333368691143
ay_dyn[9] = 11.69414846193848
ay_dyn[9] = 51.14451848753903
ay_dyn[9] = 46.602243241528825
ay_dyn[9] = 42.457325915591454
ay_dyn[9] = 38.67253615805603
ay_dyn[9] = 35.22803046624663
ay_dyn[9] = 32.1028626858346
ay_dyn[9] = 29.20788793566458
ay_dyn[9] = 26.564725331497627
ay_dyn[9] = 24.103009517751673
ay_dyn[9] = 21.825031466129694
ay_dyn[9] = 19.687294359699372
ay_dyn[9] = 17.726387754204787
ay_dyn[9] = 15.884236563855191
ay_dyn[9] = 14.14

In [ ]:
dt = zeros(N-1)
for k = 1:N-1
    @show dt[k] = Z_sol[20 + 20 * (k-1)]
end

In [12]:
# display(Z_sol)
writedlm("data_no_optimal_solution_found_5.csv", Z_sol, ',')